# Berechnung der Durchschnittlichen Spiellänge mittels Markov-Ketten
### Import der benötigten Bibliotheken

In [2]:
import sympy as sy


### Definition der Wahrscheinlichkeiten der einzelnen Würfelwerte


In [3]:
P_4D2 = {
    "dice": sy.nsimplify(1/16, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(3/8, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(1/16, tolerance=0.001, rational=True)
}

### Definition der Übergangsmatrizen

In [4]:
def RundenMatrix(dice, diceI, diceII, diceIII, diceIV):
    return sy.Matrix(
        [[dice,  diceI,  diceII,  diceIII,  diceIV * dice,  diceIV * diceI,  diceIV * diceII,  diceIV * diceIII,  diceIV * diceIV * dice,  diceIV * diceIV * diceI,  diceIV * diceIV * diceII,  diceIV * diceIV * diceIII,  diceIV * diceIV * diceIV, 0, 0, 0],
         [0,  dice,  diceI,  diceII,  diceIII * dice,  diceIV + diceIII * diceI,  diceIII * diceII,  diceIII * diceIII,  diceIII * diceIV *
            dice,  diceIII * diceIV * diceI,  diceIII * diceIV * diceII,  diceIII * diceIV * diceIII,  diceIII * diceIV * diceIV, 0, 0, 0],
            [0, 0,  dice,  diceI,  diceII * dice,  diceIII + diceII * diceI,  diceIV + diceII * diceII,  diceII * diceIII,  diceII * diceIV *
                dice,  diceII * diceIV * diceI,  diceII * diceIV * diceII,  diceII * diceIV * diceIII,  diceII * diceIV * diceIV, 0, 0, 0],
            [0, 0, 0,  dice,  diceI * dice,  diceII + diceI * diceI,  diceIII + diceI * diceII,  diceIV + diceI * diceIII,  diceI *
                diceIV * dice,  diceI * diceIV * diceI,  diceI * diceIV * diceII,  diceI * diceIV * diceIII,  diceI * diceIV * diceIV, 0, 0, 0],
            [0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII,  diceIV * dice,  diceIV *
                diceI,  diceIV * diceII,  diceIV * diceIII,  diceIV * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII * dice,  diceIV + diceIII *
                diceI,  diceIII * diceII,  diceIII * diceIII,  diceIII * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0,  dice,  diceI,  diceII * dice,  diceIII + diceII * diceI,
                diceIV + diceII * diceII,  diceII * diceIII,  diceII * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0,  dice,  diceI * dice,  diceII + diceI * diceI, diceIII +
                diceI * diceII,  diceIV + diceI * diceIII,  diceI * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,
                diceII,  diceIII,  diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,
                diceII,  diceIII,  diceIV, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII,
                diceIV*(dice + diceII + diceIII + diceIV),  diceIV * diceI],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII *
                (dice + diceII + diceIII + diceIV),  diceIV + (diceIII * diceI)],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice + diceIV,  diceI,  diceII *
                (dice + diceII + diceIII + diceIV),  diceIII + (diceII * diceI)],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice + diceIII + diceIV,
                diceI*(dice + diceII + diceIII + diceIV),  diceII + (diceI * diceI)],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                dice + diceII + diceIII + diceIV,  diceI],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])


def SchritteMatrix(dice, diceI, diceII, diceIII, diceIV):
    return sy.Matrix([
        [dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII,  diceIV, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII,  diceIV, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII, diceIV],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice +
            diceIV,  diceI,  diceII,  diceIII],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            dice+diceIII + diceIV,  diceI,  diceII],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            dice+diceII + diceIII + diceIV,  diceI],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            dice + diceI + diceII + diceIII + diceIV],
    ])


def SchritteMatrixNichtGenauInsZiel(dice, diceI, diceII, diceIII, diceIV):
    return sy.Matrix([
        [dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII,  diceIV, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII,  diceIV, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII ,diceIV],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice, diceI,  diceII, diceIII +diceIV],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII+ diceIII+ diceIV],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI+  diceII+ diceIII+ diceIV],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice+  diceI+  diceII+ diceIII+ diceIV], ])


### Festlegen welcher Würfel verwendet wird

In [5]:
M_S = SchritteMatrix(**P_4D2)
M_R = RundenMatrix(**P_4D2)
# M_S = SchritteMatrix(**D4)
# M_R = RundenMatrix(**D4)


### Überprüfung ob die Zeilensumme jeweils 1 ist

In [6]:
print("Zeilensumme Schritte Matrix:", [sum(M_S.row(i)) for i in range(16)])
print("Zeilensumme Runden Matrix:",[sum(M_R.row(i)) for i in range(16)])

Zeilensumme Schritte Matrix: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Zeilensumme Runden Matrix: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Löschen der letzten Spalte
Achtung findet inplace statt!

In [7]:
M_S.col_del(-1)
M_R.col_del(-1)


### Löschen der letzten Zeile

Achtung findet inplace statt!

In [8]:
M_S.row_del(-1)
M_R.row_del(-1)


### Definition der 15x15 - Einheitsmatrix

In [9]:
I = sy.eye(15)

### Berechnung der Ergebnismatrizen

In [10]:
E_S = (I-M_S)**-1
E_R = (I-M_R)**-1

### Berechnng der jeweils ersten Zeilensumme

In [11]:
v=sy.Matrix([[1]+[0]*14])
ZS_S= sum(v*E_S)
ZS_R= sum(v*E_R)


## Ausgabe der Ergebnisse

In [12]:
print("Schritte gerundet:", float(ZS_S))
print("Schritte exakt für Latex:", sy.latex(ZS_S))
print("Schritte exakt:")

ZS_S




Schritte gerundet: 9.867846767445288
Schritte exakt für Latex: \frac{1280316980941868}{129746337890625}
Schritte exakt:


1280316980941868/129746337890625

In [13]:
print("Runden gerundet:", float(ZS_R))
print("Runden exakt für Latex:", sy.latex(ZS_R))
print("Runden exakt:")
ZS_R

Runden gerundet: 8.391927856222578
Runden exakt für Latex: \frac{1088821907187203}{129746337890625}
Runden exakt:


1088821907187203/129746337890625

In [14]:
print("Schritte/Runden gerundet:", float(ZS_S/ZS_R))
print("Schritte/Runden exakt für Latex:", sy.latex(ZS_S/ZS_R))
print("Schritte/Runden exakt:")
ZS_S/ZS_R


Schritte/Runden gerundet: 1.175873641493274
Schritte/Runden exakt für Latex: \frac{1280316980941868}{1088821907187203}
Schritte/Runden exakt:


1280316980941868/1088821907187203

## Berechnung für die Markov-Kette wenn das Ziel nicht genau getroffen werden muss


In [15]:

M_U = SchritteMatrixNichtGenauInsZiel(**P_4D2)


In [16]:
print("Zeilensumme Schritte Matrix:", [sum(M_U.row(i)) for i in range(16)])


Zeilensumme Schritte Matrix: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [17]:
M_U.col_del(-1)
M_U.row_del(-1)

In [18]:
I = sy.eye(15)
E_U = (I-M_U)**-1
v=sy.Matrix([[1]+[0]*14])
ZS_U= sum(v*E_U)

In [19]:
print("Schritte gerundet:", float(ZS_U))
print("Schritte exakt für Latex:", sy.latex(ZS_U))
print("Schritte exakt:")

ZS_U


Schritte gerundet: 7.874997997960489
Schritte exakt für Latex: \frac{3448413510068397376}{437893890380859375}
Schritte exakt:


3448413510068397376/437893890380859375

# Vergleichswerte mit verschiedenen Würfeln

In [20]:
def berechneRunden(würfel):
    M_R = RundenMatrix(**würfel)
    # print("Zeilensumme Runden Matrix:", [sum(M_R.row(i)) for i in range(16)])
    M_R.col_del(-1)
    M_R.row_del(-1)
    I = sy.eye(15)
    E_R = (I-M_R)**-1
    v = sy.Matrix([[1]+[0]*14])
    ZS_R = sum(v*E_R)
    return ZS_R


In [21]:
def berechneSchritte(würfel):
    M_S = SchritteMatrix(**würfel)
    # print("Zeilensumme Schritte Matrix:", [sum(M_S.row(i)) for i in range(16)])
    M_S.col_del(-1)
    M_S.row_del(-1)
    I = sy.eye(15)
    E_S = (I-M_S)**-1
    v = sy.Matrix([[1]+[0]*14])
    ZS_S = sum(v*E_S)
    return ZS_S


In [22]:
def ausgabe(name, schritte, runden):
    # print(name)
    # print("Runden:", runden," = ", float(runden))
    # print("Schritte:", schritte," = ", float(schritte))
    # print(name,"&$", sy.latex(schritte),"$&$", sy.latex(runden),"$\\\\")
    # print(name,"&$", float(schritte),"$&$", float(runden),"$\\\\")
    print("{}&${:.3f}$&${:.3f}$\\\\".format(name, float(schritte), float(runden)))


In [23]:
P_D4 = {
    "dice": sy.nsimplify(0, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(1/4, tolerance=0.001, rational=True),
}
P_D2 = {
    "dice": sy.nsimplify(1/2, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(1/2, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(0, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(0, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(0, tolerance=0.001, rational=True),
}
P_2D2 = {
    "dice": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(2/4, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(0, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(0, tolerance=0.001, rational=True),
}
P_3D2 = {
    "dice": sy.nsimplify(1/8, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(3/8, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(3/8, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(1/8, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(0, tolerance=0.001, rational=True),
}
P_3D2_0TO4 = {
    "dice": sy.nsimplify(0, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(3/8, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(3/8, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(1/8, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(1/8, tolerance=0.001, rational=True),
}


In [24]:
ausgabe("D4",berechneSchritte(P_D4), berechneRunden(P_D4))
ausgabe("D2",berechneSchritte(P_D2), berechneRunden(P_D2))
ausgabe("2D2",berechneSchritte(P_2D2), berechneRunden(P_2D2))
ausgabe("3D2",berechneSchritte(P_3D2), berechneRunden(P_3D2))
ausgabe(r"3D2 0$\to$4", berechneSchritte(P_3D2_0TO4), berechneRunden(P_3D2_0TO4))
ausgabe("4D2",berechneSchritte(P_4D2), berechneRunden(P_4D2))


D4&$8.798$&$7.349$\\
D2&$30.000$&$27.000$\\
2D2&$15.750$&$13.497$\\
3D2&$11.444$&$9.660$\\
3D2 0$\to$4&$9.256$&$7.613$\\
4D2&$9.868$&$8.392$\\


In [25]:
[berechneSchritte(d) for d in [P_D2, P_2D2, P_3D2, P_4D2, P_3D2_0TO4, P_D4]]


[30,
 75331762/4782969,
 1108840853928/96889010407,
 1280316980941868/129746337890625,
 417425672509/45097156608,
 9225465/1048576]

In [26]:
[berechneRunden(d) for d in [P_D2, P_2D2, P_3D2, P_4D2, P_3D2_0TO4, P_D4]]


[27,
 88553/6561,
 1871893342281/193778020814,
 1088821907187203/129746337890625,
 10986162935393/1443109011456,
 61648355/8388608]